# 第2章 OpenAIのチャットAPIの基礎

## 設定

In [1]:
import json
import os
import time

from dotenv import load_dotenv
dotenv_path = "../.env"
load_dotenv(dotenv_path)

True

## 通常のチャット

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": "こんにちは！私はジョンと言います！"
    },
    {
        "role": "assistant",
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？"
    },
    {
        "role": "user",
        "content": "私の名前が分かりますか？"
    },
]

In [5]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(response.to_json())

{
  "id": "chatcmpl-AlYbC3jbxziDGWuQ6vvrwzio8sc6U",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "はい、ジョンさんという名前だとおっしゃっていました！何か他にお話ししたいことはありますか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1735897778,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0aa8d3e20b",
  "usage": {
    "completion_tokens": 30,
    "prompt_tokens": 69,
    "total_tokens": 99,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


## ストリーミング応答

In [6]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True, # ストリーミング応答モードを設定
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)
        time.sleep(0.1) # 本来不要だが、挙動確認のために追加

はい、あなたの名前はジョンさんですね！他に何かお話ししたいことや質問がありますか？

## JSONモード

In [7]:
messages = [
    {
        "role": "system",
        "content": "人物一覧を次のJSON形式で出力してください。 {\"people\": [\"aaa\", \"bbb\"]}", # プロンプトにJSONと含める
    },
    {
        "role": "user",
        "content": "昔々あるところにおじいさんとおばあさんがいました",
    },
]

In [8]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    response_format={"type": "json_object"}, # JSONモードを設定
)
print(response.choices[0].message.content)

{"people": ["おじいさん", "おばあさん"]}


## Function calling

### 関数設定

In [9]:
# 関数を定義（実処理）
def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps(
            {"location": "Tokyo", "temperature": "10", "unit": unit}
        )
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps(
            {"location": "Paris", "temperature": "22", "unit": unit}
        )
    else:
        return json.dumps(
            {"location": location, "temperature": "unknown"}
        )

In [10]:
# 関数を定義（LLMへの登録用）
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "現時点の各地の天気を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "英語表記の都市名, e.g. Tokyo, San Francisco",
                    },
                    "unit": {
                        "type": "string",
                        "description": "温度の単位",
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["location"],
            },
        },
    }
]

### プロンプト

In [11]:
messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

In [12]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools, # 関数を登録
)
print(response.to_json())

{
  "id": "chatcmpl-AlYbGqpDWrblskxCXYgZyIjqEPTB8",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_OiCyLQhFEfjlcw4wPe1EOMN1",
            "function": {
              "arguments": "{\"location\":\"Tokyo\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1735897782,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_5f20662549",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 95,
    "total_tokens": 111,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
 

In [13]:
# messagesに応答結果を追加
response_message = response.choices[0].message
messages.append(response_message.to_dict())
messages

[{'role': 'user', 'content': '東京の天気はどうですか？'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_OiCyLQhFEfjlcw4wPe1EOMN1',
    'function': {'arguments': '{"location":"Tokyo"}',
     'name': 'get_current_weather'},
    'type': 'function'}]}]

### 関数実行

In [14]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 複数関数が呼び出されることを考慮してループ
for tool_call in response_message.tool_calls:
    # 関数を実行
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # messagesに関数の実行結果を追加
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )
messages

{"location": "Tokyo", "temperature": "10", "unit": null}


[{'role': 'user', 'content': '東京の天気はどうですか？'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_OiCyLQhFEfjlcw4wPe1EOMN1',
    'function': {'arguments': '{"location":"Tokyo"}',
     'name': 'get_current_weather'},
    'type': 'function'}]},
 {'tool_call_id': 'call_OiCyLQhFEfjlcw4wPe1EOMN1',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "Tokyo", "temperature": "10", "unit": null}'}]

### 関数実行を踏まえた応答

In [15]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(response.choices[0].message.content)

現在、東京の気温は10度です。天気に関する詳細情報は最新の天気予報をご確認ください。
